In [2]:
from sqlalchemy import create_engine, Column, Integer, String, Date, DECIMAL, ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy.dialects.mysql import DECIMAL
import random
import datetime
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Table, MetaData

In [ ]:
username = "root"
password = "*******"
host = "localhost"
database = "new_schema"

DATABASE_URL = f"mysql+pymysql://{username}:{password}@{host}/{database}"
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()


Base = declarative_base()
metadata = MetaData()
metadata.reflect(bind=engine)
tables_to_delete = ['orderdetails', 'orders', 'products', 'customers']

for table_name in tables_to_delete:
    if table_name in metadata.tables:
        table = metadata.tables[table_name]
        table.drop(engine, checkfirst=True)

In [4]:
class Customer(Base):
    __tablename__ = 'Customers'

    customer_id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String(50))
    last_name = Column(String(50))
    email = Column(String(100))
    sign_up_date = Column(Date)
    region = Column(String(50))

    orders = relationship("Order", back_populates="customer")

class Product(Base):
    __tablename__ = 'Products'

    product_id = Column(Integer, primary_key=True, autoincrement=True)
    product_name = Column(String(100))
    category = Column(String(50))
    price = Column(DECIMAL(10, 2))

    order_details = relationship("OrderDetail", back_populates="product")

class Order(Base):
    __tablename__ = 'Orders'

    order_id = Column(Integer, primary_key=True, autoincrement=True)
    customer_id = Column(Integer, ForeignKey('Customers.customer_id'))
    order_date = Column(Date)
    order_status = Column(String(20))
    total_amount = Column(DECIMAL(10, 2))

    customer = relationship("Customer", back_populates="orders")
    order_details = relationship("OrderDetail", back_populates="order")

class OrderDetail(Base):
    __tablename__ = 'OrderDetails'

    order_id = Column(Integer, ForeignKey('Orders.order_id'), primary_key=True)
    product_id = Column(Integer, ForeignKey('Products.product_id'), primary_key=True)
    quantity = Column(Integer)

    order = relationship("Order", back_populates="order_details")
    product = relationship("Product", back_populates="order_details")

Base.metadata.create_all(engine)

In [5]:
first_names = ['John', 'Anna', 'Tom', 'Maria', 'Steve', 'Julia']
last_names = ['Smith', 'Doe', 'Brown', 'Miller', 'Johnson', 'Davis']
regions = ['North', 'South', 'East', 'West']

customers = []
for _ in range(1000):
    first = random.choice(first_names)
    last = random.choice(last_names)
    email = f"{first.lower()}.{last.lower()}@example.com"
    sign_up_date = datetime.date.today() - datetime.timedelta(days=random.randint(0, 1825)) 
    region = random.choice(regions)

    customer = Customer(
        first_name=first,
        last_name=last,
        email=email,
        sign_up_date=sign_up_date,
        region=region
    )
    customers.append(customer)

session.add_all(customers)
session.commit()

In [6]:
first_names = ['John', 'Anna', 'Tom', 'Maria', 'Steve', 'Julia']
last_names = ['Smith', 'Doe', 'Brown', 'Miller', 'Johnson', 'Davis']
regions = ['North', 'South', 'East', 'West']

customers = []
for _ in range(1000):
    first = random.choice(first_names)
    last = random.choice(last_names)
    email = f"{first.lower()}.{last.lower()}@example.com"
    sign_up_date = datetime.date.today() - datetime.timedelta(days=random.randint(0, 1825)) 
    region = random.choice(regions)

    customer = Customer(
        first_name=first,
        last_name=last,
        email=email,
        sign_up_date=sign_up_date,
        region=region
    )
    customers.append(customer)

session.add_all(customers)
session.commit()


In [7]:
products = []
categories = ['Electronics', 'Clothing', 'Books', 'Toys']

for i in range(1, 101):
    product_name = f"Product {i}"
    category = random.choice(categories)
    price = round(random.uniform(5.00, 1000.00), 2)

    product = Product(
        product_name=product_name,
        category=category,
        price=price
    )
    products.append(product)

session.add_all(products)
session.commit()


In [9]:
all_products = session.query(Product).all()

for _ in range(2500):
    customer_id = random.randint(1, 1000)
    order_date = datetime.date.today() - datetime.timedelta(days=random.randint(0, 365))
    order_status = random.choice(['Completed', 'Pending', 'Canceled'])
    total_amount = 0.0

    order = Order(
        customer_id=customer_id,
        order_date=order_date,
        order_status=order_status,
        total_amount=0.0
    )
    session.add(order)
    session.flush() 

    num_products = random.randint(1, 5)
    for _ in range(num_products):
        product = random.choice(all_products)
        quantity = random.randint(1, 10)
        total_amount += float(product.price) * quantity

        existing_detail = session.query(OrderDetail).filter(
            OrderDetail.order_id == order.order_id,
            OrderDetail.product_id == product.product_id
        ).first()

        if not existing_detail: 
            detail = OrderDetail(
                order_id=order.order_id,
                product_id=product.product_id,
                quantity=quantity
            )
            session.add(detail)

    order.total_amount = total_amount

session.commit()
